In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np
import time
import math


CAM_WIDTH, CAM_HEIGHT = 800, 1200
FRAME_REDUCTION = 150
SMOOTHENING = 7
CLICK_THRESHOLD = 35
EXIT_KEY = 27  # ESC

pyautogui.FAILSAFE = False
SCREEN_W, SCREEN_H = pyautogui.size()


# MEDIAPIPE SETUP

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)
mp_draw = mp.solutions.drawing_utils

# CAMERA SETUP

cap = cv2.VideoCapture(0)
cap.set(3, CAM_WIDTH)
cap.set(4, CAM_HEIGHT)

if not cap.isOpened():
    raise RuntimeError("Camera not accessible")

# UTILITY FUNCTIONS

def distance(p1, p2):
    return math.hypot(p2[0] - p1[0], p2[1] - p1[1])

def fingers_up(lm):
    if len(lm) != 21:
        return [0, 0, 0, 0, 0]

    fingers = []
    fingers.append(1 if lm[4][0] > lm[3][0] else 0)

    for tip in [8, 12, 16, 20]:
        fingers.append(1 if lm[tip][1] < lm[tip - 2][1] else 0)

    return fingers

# STATE VARIABLES

prev_x, prev_y = SCREEN_W // 2, SCREEN_H // 2
last_click_time = 0
pTime = 0

# MAIN LOOP

while True:
    success, frame = cap.read()
    if not success:
        continue

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    lm_list = []

    if results.multi_hand_landmarks:
        hand = results.multi_hand_landmarks[0]
        mp_draw.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS)

        for lm in hand.landmark:
            lm_list.append((int(lm.x * w), int(lm.y * h)))

    # Active region
    cv2.rectangle(
        frame,
        (FRAME_REDUCTION, FRAME_REDUCTION),
        (w - FRAME_REDUCTION, h - FRAME_REDUCTION),
        (255, 0, 255),
        2
    )

    # HAND LOGIC

    if lm_list:
        finger_state = fingers_up(lm_list)
        x_index, y_index = lm_list[8]
        x_middle, y_middle = lm_list[12]

        x_index = np.clip(x_index, FRAME_REDUCTION, w - FRAME_REDUCTION)
        y_index = np.clip(y_index, FRAME_REDUCTION, h - FRAME_REDUCTION)


        # MOVE MOUSE (Index only)

        if finger_state[1] == 1 and finger_state[2] == 0:
            x_screen = np.interp(x_index,
                                 (FRAME_REDUCTION, w - FRAME_REDUCTION),
                                 (0, SCREEN_W))
            y_screen = np.interp(y_index,
                                 (FRAME_REDUCTION, h - FRAME_REDUCTION),
                                 (0, SCREEN_H))

            curr_x = prev_x + (x_screen - prev_x) / SMOOTHENING
            curr_y = prev_y + (y_screen - prev_y) / SMOOTHENING

            pyautogui.moveTo(curr_x, curr_y)
            prev_x, prev_y = curr_x, curr_y

            cv2.circle(frame, (x_index, y_index), 12, (255, 0, 255), cv2.FILLED)

        # LEFT CLICK (Index + Middle)
        if finger_state[1] == 1 and finger_state[2] == 1:
            dist = distance((x_index, y_index), (x_middle, y_middle))

            if dist < CLICK_THRESHOLD and time.time() - last_click_time > 0.4:
                pyautogui.click()
                last_click_time = time.time()
                cv2.circle(
                    frame,
                    ((x_index + x_middle) // 2, (y_index + y_middle) // 2),
                    15,
                    (0, 255, 0),
                    cv2.FILLED
                )

    # FPS
 
    cTime = time.time()
    fps = 1 / (cTime - pTime) if pTime else 0
    pTime = cTime

    cv2.putText(frame, f"FPS: {int(fps)}",
                (20, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 0), 2)

    cv2.imshow("Virtual Mouse - MediaPipe", frame)

    if cv2.waitKey(1) & 0xFF == EXIT_KEY:
        break

cap.release()
cv2.destroyAllWindows()
